# Download YARA Retro matches - TitaniumCloud
This notebook offers the option of downloading multiple files matched by YARA Retroactive hunt.  
To create YARA rulesets and start YARA Retroactive hunt, use other Python methods from the ReversingLabs SDK.

### Used TitaniumCloud classes
- **YARARetroHunting** (*TCA-0319*)
- **FileDownload** (*TCA-0201*)

### Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md)

In [ ]:
import json
import os
from ReversingLabs.SDK.ticloud import YARARetroHunting, FileDownload


CREDENTIALS = json.load(open("credentials.json"))
USERNAME = CREDENTIALS.get("ticloud").get("username")
PASSWORD = CREDENTIALS.get("ticloud").get("password")

cwd = os.getcwd()
upper_level = os.path.dirname(cwd)
USER_AGENT = json.load(open(os.path.join(upper_level, "user_agent.json")))["user_agent"]

yara_retro = YARARetroHunting(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)

file_download = FileDownload(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)
        
        

### 1. Batch-download function
This function will be used for downloading each batch of matched samples. Execute it so you can use it further on in the notebook.

In [ ]:
def download_batch(sample_list, download_path):
    for sample in sample_list:
        response = file_download.download_sample(hash_input=sample.get("sha1"))
        
        with open(os.path.join(download_path, sample.get("sha1")), "wb") as file_handle:
            file_handle.write(response.content)


### 2. Main function
To retrieve batches of samples matched by a YARA Retro ruleset and download them as files, use the `download_retro_hunt_matches` function.  
This function accepts the following parameters:
- `time_format`: defines the format of the used time value. possible options are 'utc' and 'timestamp'
- `time_value`: point back in time from which the retro hunt will start producing matches
- `download_path`: needs to be a full path to the existing folder

Execute the function so you can use it multiple times.

In [ ]:
def download_retro_hunt_matches(time_format, time_value, download_path):
    response = yara_retro.yara_retro_matches_feed(
        time_format=time_format,
        time_value=time_value
    )
    
    resp_json = response.json()
    entries = resp_json.get("rl", {}).get("feed", {}).get("entries", [])
    
    download_batch(sample_list=entries, download_path=download_path)

### 3. Run

In [ ]:
download_retro_hunt_matches(
    time_format="timestamp",
    time_value="1734994735",
    download_path="/change/me"
)